1. 라이브러리 설치

In [ ]:
!pip install --upgrade google.cloud.aiplaform

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

2. 환경변수 설정

In [ ]:
import vertexai

PROJECT_ID=!gcloud config get-value project
PROJECT_ID=PROJECT_ID[0]
LOCATION="us-east1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

3. 모델 활용

3.1 모델 생성

In [ ]:
from vertexai.generative_models import (
    GenerativeModel,
    Part,
)

model = GenerativeModel("gemini-1.5-flash")

3.2 video 분석

In [ ]:
from IPython.display import Video

file_path="github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
video_uri = f"gs://{file_path}"
video_url=f"https://storage.googleapis.com/{file_path}"

Video(video_url, width=450)

In [ ]:
prompt="""
영상을 보고 다음 질문에 답하시오.
1. 주인공의 직업은 무엇인가?
2. 핸드폰의 주 기능은 무엇인가?
3. 촬영한 도시는 어디인가?

답변은 JSON으로 표현하시오.
"""

video=Part.from_uri(video_uri, mime_type="video/mp4")

In [ ]:
contents=[prompt, video]

response = model.generate_content(contents)

print(response.text)

```json
{
 "주인공의 직업": "사진작가",
 "핸드폰의 주 기능": "야경 모드",
 "촬영한 도시": "도쿄"
}
```


In [ ]:
contents=[video, prompt]

response = model.generate_content(contents)

print(response.text)

```json
{
 "1": "사진 작가",
 "2": "야간 모드",
 "3": "도쿄"
}
```


3.3 schema 적용

In [ ]:
response_schema={
    "type": "ARRAY",
    "items":{
        "type": "OBJECT",
        "properties":{
            "question": {"type": "STRING", "description": "Summary in noun form"},
            "answer": {"type": "STRING", "description": "Summary in noun form"}
        },
        "required": ["question", "answer"]
    }
}

In [ ]:
prompt="""
영상을 보고 다음 질문에 답하시오.
1. 주인공의 직업은 무엇인가?
2. 핸드폰의 주 기능은 무엇인가?
3. 촬영한 도시는 어디인가?
"""
content=[video, prompt]

response = model.generate_content(content)
print(response.text)

1. 주인공은 사진 작가입니다.
2. 핸드폰의 주 기능은 야경 모드입니다.
3. 촬영한 도시는 도쿄입니다.


In [ ]:
from vertexai.generative_models import GenerationConfig

content=[video, prompt]

response = model.generate_content(
    content,
    generation_config=GenerationConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    )
)
print(response.text)

[{"question": "주인공의 직업", "answer": "사진작가"}, {"question": "핸드폰의 주 기능", "answer": "야경모드"}, {"question": "촬영한 도시", "answer": "도쿄"}]


In [ ]:
from vertexai.generative_models import GenerationConfig

content=[prompt, video]

response = model.generate_content(
    content,
    generation_config=GenerationConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    )
)
print(response.text)

[{"question": "주인공의 직업", "answer": "사진작가"}, {"question": "핸드폰의 주 기능", "answer": "야경 모드"}, {"question": "촬영한 도시", "answer": "도쿄"}]
